# Capturing Places and Persons

## This notebook processes text from music personalities' biographies and extract historical meetups information 
 Pre-requirements:
 Entity annotations after executing "02_queryDbpedia.ipynb" Notebook

 For each file with entity annotations
 - Read files from cacheSpotlightResponse/
 - Identify People entities:
   - "http://dbpedia.org/ontology/Person"
   - "http://dbpedia.org/ontology/MusicalArtist"
 - Identify Place entities:
   - 'http://dbpedia.org/ontology/Place'
 - When an entity has an empty type
   - Read corresponding emptyEntity response from DBpedia in cacheSpotlightResponse/ directory
 - Store annotations in extractedEntitiesPersonPlaceOnly/

 Directories information:
 - cacheSpotlightResponse/ : collection of biographies in CSV format. Each biography contains the list of entities identified using DBpedia Spotlight, each linked to its corresponding sentence
 - extractedEntitiesPersonPlaceOnly/ : response from DBpedia Spotlight entity annotation grouped by biography

In [1]:
import json
import os
import pandas as pd
from _datetime import date
import time
from operator import itemgetter

# For DBpedia spotlight, PPE entities
import requests
import pycurl
from urllib.request import urlopen
from urllib.parse import quote

In [2]:
# extract only the ones that do not exist in folder
files_list = [f for f in os.listdir('extractedEntitiesPersonPlaceOnly') if not f.startswith('.')]
# parse to dataframe
df_query = pd.DataFrame(files_list, columns=['file_name'])
print("Extracted PP: ",len(df_query))
# Read file with the 33k biographies
bioListMEETUPS_df = pd.read_csv('TOTAL_download_biographies.csv')
bioListMEETUPS_df.rename(columns={'id': 'file_name','s':'resource'}, inplace=True)
bioListMEETUPS_df['file_name'] = bioListMEETUPS_df['file_name'].astype(str) + '.csv'
print("Total biographies: ",len(bioListMEETUPS_df))
df_result = bioListMEETUPS_df[~bioListMEETUPS_df['file_name'].isin(df_query['file_name'])]
print("Left to process: ",len(df_result))
df_result.to_csv('totalBiosToProcessPP.csv',index=False)
df_result.info()
df_result.head()

Extracted PP:  0
Total biographies:  33309
Left to process:  33309
<class 'pandas.core.frame.DataFrame'>
Int64Index: 33309 entries, 0 to 33308
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   resource   33309 non-null  object
 1   file_name  33309 non-null  object
dtypes: object(2)
memory usage: 780.7+ KB


,resource,file_name
0,http://dbpedia.org/resource/Abbe_Lane,2205692.csv
1,http://dbpedia.org/resource/Jimena_Barón,17441602.csv
2,http://dbpedia.org/resource/Guy_Lafitte,10925966.csv
3,http://dbpedia.org/resource/Dominique_Jameux,54550593.csv
4,http://dbpedia.org/resource/Phil_Berg_(talent_...,65723345.csv


In [3]:
biospp_df_1 = df_result.iloc[:11103]
biospp_df_2 = df_result.iloc[11103:22206]
biospp_df_3 = df_result.iloc[22206:]

biospp_df_1.to_csv('totalBiosToProcessPP_1.csv',index=False)
biospp_df_2.to_csv('totalBiosToProcessPP_2.csv',index=False)
biospp_df_3.to_csv('totalBiosToProcessPP_3.csv',index=False)

In [7]:
# USING ONLY FOR SAMPLING - 1002
files_list = [f for f in os.listdir('extractedEntities') if not f.startswith('.')]
# parse to dataframe
df_query = pd.DataFrame(files_list, columns=['file_name'])
df_query.to_csv('totalBiographiesEntities.csv',index=False)

## 1. Process DBpedial Spotlight entity annotation: functions

In [2]:
def executeQueryDbpedia(q, f='application/json'):
    epr = "http://dbpedia.org/sparql"
    try:
        params = {'query': q}
        resp = requests.get(epr, params=params, headers={'Accept': f})
        # return resp.text
        return resp
    except Exception as e:
        # print(e, file=sys.stdout)
        if hasattr(e, 'message'):
            print(e.message)
        else:
            print(e)
        raise
        
# retrieve entities information when they are not in cache
def queryEntityLeft(uri,item):
    # retrieve the next id to store the empty entitites
    df_master = pd.read_csv('cacheSpotlightResponse/emptyTypes_master.csv')
    df_master['id'] = df_master['file_name'].str.replace('emptyTypes_','')
    df_master['id'] = df_master['id'].str.replace('.csv','')
    df_master['id'] = df_master['id'].astype(str).astype(int)
    df_master = df_master.sort_values(by='id', ascending=False)
    last_file_id = df_master['id'].loc[df_master.index[0]]
    last_file_id = int(last_file_id)+1
    # last_file_id +=1
    
    df_results = pd.DataFrame()
    query_text = "SELECT * WHERE { <" + uri + "> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type>  ?o }"
    try:
        # Execute query against sparql endpoint, query types
        results = executeQueryDbpedia(query_text).json()
        # print(results)
        # if query returns a response
        if 'results' in results:
            # to obtain the list of types
            res_1 = list(map(itemgetter('o'), results['results']['bindings']))
            res_2 = list(map(itemgetter('value'), res_1))

            df_results['types'] = res_2
            df_results['URI'] = item.URI
            df_results['entity'] = item.entity
            df_results['support'] = item.support
            df_results['offset'] = item.offset
            df_results['similarityScore'] = item.similarityScore
            df_results['percentageOfSecondRank'] = item.percentageOfSecondRank

            df_results['sentenceIndex']=item.sentenceIndex
            df_results['paragraphIndex'] = item.paragraphIndex
            df_results['section'] = item.section

            df_new_master_row = pd.DataFrame({'URI':[item.URI],'entity':[item.entity],
                                                          'file_name':['emptyTypes_{}.csv'.format(str(last_file_id))]})
            df_new_master_row.to_csv('cacheSpotlightResponse/emptyTypes_master.csv',mode='a',
                                 index=False,header=False)
            # print("Saved master file: " + str(last_file_id) + ". Len: " + str(len(df_new_master_row)))

            file_exists = os.path.isfile('cacheSpotlightResponse/emptyTypes_'+str(last_file_id)+'.csv')
            if not file_exists:
                df_results.to_csv('cacheSpotlightResponse/emptyTypes_'+str(last_file_id)+'.csv',index=False)
            else:
                df_results.to_csv('cacheSpotlightResponse/emptyTypes_'+str(last_file_id)+'.csv',mode='a',
                                 index=False,header=False)
    except Exception as ex:
        print("Blank type: ****")
        if hasattr(ex, 'message'):
            print(ex.message)
        else:
            print(ex)
        # print(ex, file=sys.stdout)
    return df_results

In [5]:
# Create error file for later review
df_error = pd.DataFrame(columns=['file_name'])
df_error.to_csv('temp_error.csv',index=False)

df_error.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   file_name  0 non-null      object
dtypes: object(1)
memory usage: 0.0+ bytes


## 1. Extract entities: People, places

In [3]:
# If the entity is classified as person return True
# include a validation to identify other types such as 
# dbo:Composer, Tenor that are not persons
# http://dbpedia.org/resource/Contralto
# False
# http://dbpedia.org/resource/Edward_Elgar
# True
# http://dbpedia.org/resource/Composer
# dct:subject dbc:Idioms
# http://dbpedia.org/resource/Bassoon rdf:type dbo:Person false

# call function to check if entity is a place
# Input: URI of candidate entity
# Output: boolean value, False it is not a place, True if it is a place
def checkPlaceEntity(URI_string, entTypes_df):
    # it is always False unless the query returns True
    item = ''
    response = False
    entitiesPlace_df = readCachedFilePlaceEntity()

    localEntitiesPlace_df = entitiesPlace_df.loc[(entitiesPlace_df['URI'] == URI_string)]
    if len(localEntitiesPlace_df) == 1:
        # It is not a person
        return localEntitiesPlace_df['response'].loc[localEntitiesPlace_df.index[0]],localEntitiesPlace_df['type'].loc[localEntitiesPlace_df.index[0]]
    
    entitiesPlace_df = entitiesPlace_df.loc[(entitiesPlace_df['URI'] == URI_string) & (entitiesPlace_df['response'] == False)]
    if len(entitiesPlace_df) > 0:
        return False, localEntitiesPlace_df['type'].loc[localEntitiesPlace_df.index[0]]
    
    subClassesPlace_df = pd.read_csv('cacheSpotlightResponse/otherPlacesEntities.csv')
    subClassesPlace_df = subClassesPlace_df.set_index(['URI'])
    
    temp_df = subClassesPlace_df.loc[subClassesPlace_df.index.isin(entTypes_df['types'].to_list())]
    if temp_df.empty:
        return False,""
    
    if len(temp_df) >0:
        temp_df = temp_df.reset_index()
        response = True
        item = temp_df["URI"].loc[temp_df.index[0]]
        # Save the data
        datarow = pd.Series(data={'URI':URI_string,'response':response,'type':item})
        # entitiesPlace_df = entitiesPlace_df.append(datarow, ignore_index=True)
        entitiesPlace_df = pd.concat([entitiesPlace_df, datarow], ignore_index=True)
        entitiesPlace_df.to_csv('cacheSpotlightResponse/validatePlacesEntities.csv',index=False)

    return response, item

# cache the answers
def readCachedFilePlaceEntity():
    # check if the file exists
    if os.path.isfile('cacheSpotlightResponse/validatePlacesEntities.csv'):
        return pd.read_csv('cacheSpotlightResponse/validatePlacesEntities.csv')
    else:
        otherEntitiePlaces_df = pd.DataFrame(columns=['URI','response',"type"])
         # create the file to store the list of biographies with empty entities 
        otherEntitiePlaces_df.to_csv('cacheSpotlightResponse/validatePlacesEntities.csv',index=False)
    return otherEntitiePlaces_df

# call function, search all the values in DBpedia or locally, 
# returns True if the URI is of a person, False otherwise
# it is always False unless the query returns True
def checkPersonEntity(URI_string, entTypes_df):
    res = False
    
    #two steps
    # check if the person has a DOB. If it does have a DOB then it confirms it is a person and returns TRUE, otherwise
    # res = checkDOBEntity(URI_string)
    # if res:
    #     return True
    # check if the person is of one of the types in the following list.
    # if the URI is one of the triples, then it is not a Person.
    # else:
    res = checkPersonRolesEntity(URI_string, entTypes_df)
    if res:
        return False
    else:
        return True
    return res

def checkDOBEntity(URI_string):
    entitiesDOB_df = readCachedFileDOBEntity()
    # perform a validation with date of birth first
    localEntitiesDOB_df = entitiesDOB_df.loc[(entitiesDOB_df['URI'] == URI_string)]
    if len(localEntitiesDOB_df) > 0:
        # It is not a person
        return localEntitiesDOB_df['response'].loc[localEntitiesDOB_df.index[0]]

    query_text = "ASK  { <"+URI_string+"> dbo:birthDate ?birthDate }"
    results = executeQueryDbpedia(query_text).json()
        # if query returns a response
    if 'boolean' in results:
        response = results['boolean']
        # Save the data
        if response:
            datarow = pd.Series(data={'URI':URI_string,'response':True,'type':"dbo:birthDate ?birthDate"})
            # entitiesDOB_df = entitiesDOB_df.append(datarow, ignore_index=True)
            entitiesDOB_df = pd.concat([entitiesDOB_df, datarow], ignore_index=True)
            entitiesDOB_df.to_csv('cacheSpotlightResponse/otherPersonEntitiesDOB.csv',index=False)
            return True
        else:
            datarow = pd.Series(data={'URI':URI_string,'response':False,'type':"dbo:birthDate ?birthDate"})
            # entitiesDOB_df = entitiesDOB_df.append(datarow, ignore_index=True)
            entitiesDOB_df = pd.concat([entitiesDOB_df, datarow], ignore_index=True)
            entitiesDOB_df.to_csv('cacheSpotlightResponse/otherPersonEntitiesDOB.csv',index=False)
            return False
    
# call function, search all the values in DBpedia or locally, returs True if the URI is of a person function/role, False otherwise
# Input: entity URI
# Output is always True(a person fuction/role) unless the query returns False
def checkPersonRolesEntity(URI_string,entTypes_df):
    response = False
    # Verify first if we already queried DBpedia spotlight for entity validation
    # read file with list of entities
    entities_df = readCachedFileTypeEntity()
    localEntities_df = entities_df.loc[(entities_df['URI'] == URI_string) & (entities_df['response'] == True)]
    if len(localEntities_df) > 0:
        # It is not a person
        return True
    
    object_type_list = ["http://dbpedia.org/ontology/PersonFunction"]
    temp_df = entTypes_df.set_index(['types'])
    temp_df = temp_df.loc[temp_df.index.isin(object_type_list)]
    if len(temp_df) > 0:
        temp_df = temp_df.reset_index()
        response = True
        item = temp_df["types"].loc[temp_df.index[0]]
        # Save the data
        datarow = pd.Series(data={'URI':URI_string,'response':response,'type':item})
        entities_df = entities_df.append(datarow, ignore_index=True)
        entities_df.to_csv('cacheSpotlightResponse/otherPersonEntities.csv',index=False)
        return response
    
    object_dct_list = ["dct:subject dbc:Musical_terminology","dct:subject dbc:Idioms",
                  "dct:subject dbc:Orchestral_instruments", "dct:subject dbc:Concert_band_instruments", "dct:subject dbc:Occupations_in_music"]
    
    localEntities_int = len(entities_df.loc[(entities_df['URI'] == URI_string) & (entities_df['response'] == False)])
    if localEntities_int == len(object_dct_list):
        return False

    for item in object_dct_list:
        localEntities_df = entities_df.loc[(entities_df['URI'] == URI_string) & (entities_df['type'] == item)]
        if len(localEntities_df) == 0:
            # build query
            query_text = "PREFIX dct: <http://purl.org/dc/terms/> \
            ASK  { <"+URI_string+"> "+item+" }"
            print(query_text)
            # E.g.: ASK  { <http://dbpedia.org/resource/Conducting> rdf:type dbo:PersonFunction } 
            # true for composer, solicitor, conducting
            # query DBpedia
            results = executeQueryDbpedia(query_text).json()
            # if query returns a response
            if 'boolean' in results:
                # to obtain the list of types
                response = results['boolean']
                # Save the data
                datarow = pd.Series(data={'URI':URI_string,'response':response,'type':item})
                # entities_df = entities_df.append(datarow, ignore_index=True)
                entities_df = pd.concat([entities_df, datarow], ignore_index=True)
                entities_df.to_csv('cacheSpotlightResponse/otherPersonEntities.csv',index=False)
                if response:
                    break 
    return response

# cache the answers
def readCachedFileTypeEntity():
    # check if the file exists
    if os.path.isfile('cacheSpotlightResponse/otherPersonEntities.csv'):
        return pd.read_csv('cacheSpotlightResponse/otherPersonEntities.csv')
    else:
        otherEntities_df = pd.DataFrame(columns=['URI','response',"type"])
         # create the file to store the list of biographies with empty entities 
        otherEntities_df.to_csv('cacheSpotlightResponse/otherPersonEntities.csv',index=False)
    return otherEntities_df

# cache the answers
def readCachedFileDOBEntity():
    # check if the file exists
    if os.path.isfile('cacheSpotlightResponse/otherPersonEntitiesDOB.csv'):
        return pd.read_csv('cacheSpotlightResponse/otherPersonEntitiesDOB.csv')
    else:
        otherEntitieDOBs_df = pd.DataFrame(columns=['URI','response',"type"])
         # create the file to store the list of biographies with empty entities 
        otherEntitiesDOBs_df.to_csv('cacheSpotlightResponse/otherPersonEntitiesDOB.csv',index=False)
    return otherEntitiesDOBs_df

# # URI_string = "http://dbpedia.org/resource/Alan_Turing"
# # URI_string = "http://dbpedia.org/resource/Composer"
# # URI_string = "http://dbpedia.org/resource/Contralto"
# # URI_string = "http://dbpedia.org/resource/Man"
# URI_string = "http://dbpedia.org/resource/Edward_Elgar"
# role = checkPersonEntity(URI_string)
# print("Finale: ")
# print(role)

# # response, types = checkPlaceEntity(item_queriedType.URI, df_empty_type)
# response = checkPersonEntity(item_queriedType.URI, df_empty_type)
# print(response)
# # print(types)

In [4]:
# !python 021_Identify_PP.py > out.txt
# 1. Extract People and Places entities
entitiyTypes = ['DBpedia:Person','DBpedia:MusicalArtist','DBpedia:Place','DBpedia:SocietalEvent']
# Entity place types
entityPlaceTypesExt = ['http://www.wikidata.org/entity/Q41176','http://www.wikidata.org/entity/Q486972',
                      'http://dbpedia.org/ontology/Place','http://dbpedia.org/ontology/Location','http://www.wikidata.org/entity/Q6256']
entityPeopleTypesExt = ['http://dbpedia.org/ontology/Person','http://dbpedia.org/ontology/MusicalArtist',
                       'http://www.wikidata.org/entity/Q215627']
object_dct_list = ["dct:subject dbc:Musical_terminology","dct:subject dbc:Idioms","dct:subject dbc:Orchestral_instruments", 
                   "dct:subject dbc:Concert_band_instruments", "dct:subject dbc:Occupations_in_music","http://dbpedia.org/ontology/PersonFunction"]
# http://dbpedia.org/ontology/Person
# df_entityType = pd.DataFrame({'id':[1,2],'types':['http://dbpedia.org/ontology/Person',
#                                                       'http://dbpedia.org/ontology/MusicalArtist']})
# read master with index of entities
df_master_cache = pd.read_csv('cacheSpotlightResponse/emptyTypes_master.csv')

# use chunk to load a small number of files in memory
for chunk in pd.read_csv('totalBiosToProcessPP_1.csv', chunksize=50):
# for chunk in pd.read_csv('totalTest.csv', chunksize=50):
# for chunk in pd.read_csv('totalBiographiesBenchmark.csv', chunksize=30):
# for chunk in pd.read_csv('list_wikiIdSample.csv', chunksize=30):
    df_files = pd.DataFrame()
    df_files['file_name'] = chunk['file_name']
    
    # iterate over files
    for file_name_item in df_files.itertuples():
        if os.path.isfile('extractedEntitiesPersonPlaceOnly/'+file_name_item.file_name):
            print(file_name_item.file_name, " file exists.")
            continue
        try:
            # start = time.time()
            print(file_name_item.file_name)
            # check if the file exists, e.g., 10085.csv
            # file should exist, it was create during queryDBpedia process
            file_exists = os.path.isfile('cacheSpotlightResponse/'+file_name_item.file_name)

            if file_exists:
                # read the cached results from the query
                entitiesByBiography_df = pd.read_csv('cacheSpotlightResponse/'+file_name_item.file_name)
                # FILTER
                # Validate that the "SIMILARITYSCORE" given by DBpedia Spotlight is greater that 0.6
                # entitiesByBiography_df = entitiesByBiography_df.loc[entitiesByBiography_df['similarityScore'] > 0.6].copy()
                # FILTER
                # not taking into account paragraphs with external links and references
                entitiesByBiography_df = entitiesByBiography_df[~entitiesByBiography_df['section'].isin(['== References ==',
                                                                                                         '== Filmography ==','== External links =='])]
                
                # surfaceForm: entity as found in the text
                # entitiesByBiography_df.rename(columns = {'surfaceForm':'entity'},inplace = True)
                # column types contain DBpedia classes, e.g., dbo:Person, Schema:Place, etc.
                # select all rows where types <> NA, not empty = ne
                df_ne = entitiesByBiography_df.loc[~entitiesByBiography_df['types'].isna()] 

                df_result = pd.DataFrame()
                if len(df_ne) >0:
                    for entity in entitiyTypes:
                        df_temp = df_ne[df_ne['types'].str.contains(entity)].copy()

                        if not df_temp.empty:
                            if entity == 'DBpedia:Person' or entity == 'DBpedia:MusicalArtist':
                                df_temp['entType'] = 'person'
                            elif entity == 'DBpedia:Place':
                                df_temp['entType'] = 'place'
                            elif entity == 'DBpedia:SocietalEvent' or entity == 'DBpedia:Event':
                                df_temp['entType'] = 'event'

                            # df_result = df_result.append(df_temp)
                            df_result = pd.concat([df_result, df_temp], ignore_index=True)
                            
                # print("First len df_result: ",len(df_result))

                # now filter all the entities that are empty, this df contains all the info of the matching
                # e.g., surfaceForm, similarityScore, etc
                df_e = entitiesByBiography_df.loc[entitiesByBiography_df['types'].isna()] 
                
                # only if the dataframe has rows with empty value in column "entity"
                # Notebook 02 was in charge of identify empty entities, query and download to cache the real type
                if not df_e.empty:
                    # list of entities and the name of the files that store the results from the query
                    # when merging, only the entities that are stored in cache will have a file name value in column "file_name"
                    
                    df_merge = df_master_cache.merge(df_e, on=['URI','entity'],how='right')
                    
                    # df_merge.to_csv('extractedEntitiesPersonPlaceOnly/df_merge.csv',index=False)

                    # for all documents not in cache, meaning that value in column "file_name" is empty
                    df_cache = df_merge.loc[df_merge['file_name'].isna()]
                    if len(df_cache)>0:
                        # iterate over the df with all the entities that not in cache
                        for item_empty in df_cache.itertuples():
                            # query DBpedia, store results in local folder and update the master file
                            df_temp = queryEntityLeft(item_empty.URI,item_empty)
                        
                        # read the master cache again with the latest updates after checking for URIs in previous line
                        df_master_cache = pd.read_csv('cacheSpotlightResponse/emptyTypes_master.csv')
                        # merge again all the entities that had column "types" value as empty to find the name of the file
                        # where the response from DBpedia is stored
                        
                        df_merge = df_master_cache.merge(df_e, on=['URI','entity'],how='right')
                        df_merge.drop_duplicates(subset=['URI','entity'],keep='first',inplace=True)
                    # df_merge.to_csv('extractedEntitiesPersonPlaceOnly/df_merge.csv',index=False)

                    # df_cache will contain all the entities and the file_name value from where to read its URIs
                    # filter all the rows that have column "file_name" as having a value
                    df_cache = df_merge.loc[~df_merge['file_name'].isna()] 
                    # print("df_cache")
                    # print(df_cache)
                    for item_queriedType in df_cache.itertuples():
                        appendRow = False
                        typeEntity = ''

                        # read cache file
                        file_exists = os.path.isfile('cacheSpotlightResponse/'+item_queriedType.file_name)
                        if file_exists:
                            # once the file is located, filter the rows that match entity and URI
                            df_empty_type = pd.read_csv('cacheSpotlightResponse/'+item_queriedType.file_name)
                            # print("df_empty_type ",df_empty_type)
                            df_empty_type.drop_duplicates(subset=['entity','URI','types'],keep='first',inplace=True)
                            # print("df_empty_type ",df_empty_type)
                            try:
                                df_empty_type = df_empty_type.query("""entity == "{}" and URI=="{}" """.format(item_queriedType.entity,item_queriedType.URI))
                            except SyntaxError as ex:
                                # df_empty_type = df_empty_type.query("""entity == '{}' and URI=='{}'""".format(item_queriedType.entity,item_queriedType.URI))
                                df_empty_type = df_empty_type[df_empty_type['URI'].str.contains(item_queriedType.URI)].copy()
                            except TokenError as te:
                                df_empty_type = df_empty_type.query("entity == '{}' and URI=='{}'".format(item_queriedType.entity,item_queriedType.URI))
                                    
                            # print("df_empty_type AFTER filtering",df_empty_type)
                                # # filter the rows that match entity and URI
                            # df_empty_place = df_empty_type[df_empty_type['types'].isin(entityPlaceTypesExt)]
                            if not df_empty_type.empty:
                                # filter the rows that match entity and URI
                                df_empty_place = df_empty_type[df_empty_type['types'].isin(entityPlaceTypesExt)].copy()

                                # Classify entities according to type
                                # First, identify entity types Place, filter all rows with column "types" value equal to Place URI
                                # df_temp = df_empty_type[df_empty_type['types'].str.contains('http://dbpedia.org/ontology/Place')].copy()
                                # df_temp = df_empty_type.query('types == "http://dbpedia.org/ontology/Place"')
                                # if not df_temp.empty:
                                # response, types = checkPlaceEntity(item_queriedType.URI, df_empty_type)
                                # if response:
                                    # Before adding this entity to the list, verify if it is a place, by asking about alternative triples
                                    # E.g. having a capital
                                    # response = checkPlaceEntity(item_queriedType.URI)
                                    # types = df_temp['types'].loc[df_temp.index[0]]
                                    # types = "DBpedia:Place,DBpedia:Country"
                                    # types = 
                                if not df_empty_place.empty:
                                    # print(df_empty_place)
                                    types = "DBpedia:Place,DBpedia:Location,wikidata:Q486972,wikidata:Q41176"
                                    entType = 'place'
                                    #appendRow = True
                                    #if appendRow:
                                    df_new_row = pd.DataFrame([[item_queriedType.URI, item_queriedType.support, types, item_queriedType.entity, item_queriedType.offset, 
                                                                item_queriedType.similarityScore,item_queriedType.percentageOfSecondRank, item_queriedType.sentence, 
                                                                item_queriedType.sentenceIndex, item_queriedType.paragraphIndex, 
                                                               item_queriedType.section]],columns=list(entitiesByBiography_df))
                                    df_new_row['entType'] = entType
                                    # print("df_new_row: ", df_new_row)
                                    # df_result = df_result.append(df_new_row,ignore_index=True)
                                    df_result = pd.concat([df_result, df_new_row], ignore_index=True)
                                    # print("Len df_result after adding places: ",len(df_result))
                                    appendRow = False
                                    continue

                                # Second, identify entity types Person, filter all rows with column "types" value equal to Person URI
                                df_empty_people = df_empty_type[df_empty_type['types'].isin(entityPeopleTypesExt)].copy()
                                
                                # df_temp = df_empty_type.query('types == "http://dbpedia.org/ontology/Person" or types=="http://dbpedia.org/ontology/MusicalArtist"')
                                # df_temp = df_temp.sort_values(by='types', ascending=False)

                                if not df_empty_people.empty:
                                    # print(df_empty_people)
                                    # specific types that do not refer to people
                                    if len(df_empty_type[df_empty_type['types'].isin(object_dct_list)])>0:
                                        
                                        continue
                                    # Before adding this entity to the list, review that it is a person and it is not a PersonFunction or MusicalTerminology
                                    # call function, search all the values in DBpedia, returs True if the URI is a person, False otherwise
                                    # response = checkPersonEntity(item_queriedType.URI,df_empty_type)

                                    # if response:
                                    # types = df_temp['types'].loc[df_temp.index[0]]
                                    types = "DBpedia:Person,DBpedia:MusicalArtist,wikidata:Q215627"
                                    entType = 'person'
                                    #     appendRow = True
                                    # else:
                                    #     appendRow = False
                                    # if appendRow:
                                    df_new_row = pd.DataFrame([[item_queriedType.URI, item_queriedType.support, types, item_queriedType.entity, item_queriedType.offset, 
                                                                item_queriedType.similarityScore,item_queriedType.percentageOfSecondRank, item_queriedType.sentence, 
                                                                item_queriedType.sentenceIndex, item_queriedType.paragraphIndex, 
                                                               item_queriedType.section]],columns=list(entitiesByBiography_df))
                                    df_new_row['entType'] = entType
                                    # print("df_new_row: ", df_new_row)
                                    # df_result = df_result.append(df_new_row,ignore_index=True)
                                    df_result = pd.concat([df_result, df_new_row], ignore_index=True)
                                    # print("Len df_result after adding people: ",len(df_result))

                                    appendRow = False

                    # print('No empty types')
            # IF file does not exists, then the 
            else:
                print("[INFO] - File does not exist, run 020_queryDbpedia book. File: " + file_name_item.file_name)

            df_result['wikiPageID'] = file_name_item.file_name.replace('.csv','')
            # delete duplicate rows
            df_result.drop_duplicates(subset=['entity','URI','offset','sentenceIndex','paragraphIndex'],keep='first',inplace=True)
            df_result.to_csv('extractedEntitiesPersonPlaceOnly/'+file_name_item.file_name,index=False)
            # end = time.time()
            # print("The time of execution of above program is :", end-start)
        except (pd.errors.ParserError, KeyError) as pe:
            print('Error: ' + file_name_item.file_name)
            if hasattr(pe, 'message'):
                print(pe.message)
            else:
                print(pe)
            df_error = pd.DataFrame([[file_name_item.file_name]],columns=['file_name'])
            df_error.to_csv('temp_error.csv',mode='a',index=False,header=False)
    time.sleep(90)

2205692.csv  file exists.
17441602.csv  file exists.
10925966.csv  file exists.
54550593.csv  file exists.
65723345.csv  file exists.
52401446.csv  file exists.
34854217.csv  file exists.
3634591.csv  file exists.
54393783.csv  file exists.
3256685.csv  file exists.
705165.csv  file exists.
6393313.csv  file exists.
18047601.csv  file exists.
32442400.csv  file exists.
1355287.csv  file exists.
39637341.csv  file exists.
31509353.csv  file exists.
25522049.csv  file exists.
3942744.csv  file exists.
43499348.csv  file exists.
15707096.csv  file exists.
2655534.csv  file exists.
23757092.csv  file exists.
6242990.csv  file exists.
7340504.csv  file exists.
762840.csv  file exists.
196224.csv  file exists.
26944201.csv  file exists.
4432855.csv  file exists.
207591.csv  file exists.
2300926.csv  file exists.
34006449.csv  file exists.
5764028.csv  file exists.
64462922.csv  file exists.
25646577.csv  file exists.
34684163.csv  file exists.
16837034.csv  file exists.
978583.csv  file exis

/tmp/ipykernel_1879/3683733935.py:21: FutureWarning: The default value of regex will change from True to False in a future version.
  df_master['id'] = df_master['id'].str.replace('.csv','')


2176016.csv
22492855.csv
10508305.csv
56007683.csv
39757273.csv
51440798.csv
1823477.csv
455138.csv
19588768.csv
8855.csv
6310838.csv
7265624.csv
703488.csv
18778044.csv
1530914.csv
5138490.csv
6997149.csv
870043.csv
14947236.csv
13241032.csv
443772.csv
39396828.csv
24972730.csv
33948077.csv
17168689.csv
29285596.csv
846297.csv
34363493.csv
17885427.csv
32963865.csv
36532598.csv
46995861.csv
920660.csv
9581533.csv
2549987.csv
15374313.csv
13631044.csv
36418371.csv
50322721.csv
40365002.csv
4982329.csv
7651004.csv
15228478.csv
67501861.csv
34683790.csv
19539389.csv
46416893.csv
12721618.csv
32328278.csv
706282.csv
11381309.csv
5001211.csv
1254005.csv
38473068.csv
13974129.csv
3657182.csv
34315511.csv
23919644.csv
14080671.csv
61539916.csv
12125143.csv
9711198.csv
22121840.csv
1394553.csv
54108487.csv
54966252.csv
42386286.csv
1924456.csv
17776820.csv
1092824.csv
2592629.csv
2934315.csv
5143989.csv
37029067.csv
34546475.csv
27539399.csv
68265086.csv
33429767.csv
47128108.csv
9888835.csv


/tmp/ipykernel_1879/3683733935.py:21: FutureWarning: The default value of regex will change from True to False in a future version.
  df_master['id'] = df_master['id'].str.replace('.csv','')


4437640.csv
13984595.csv
14293844.csv
708292.csv
17055253.csv
2938274.csv
61613808.csv
16005995.csv
26566588.csv
14053372.csv
47943699.csv
16765590.csv
12869278.csv
53127398.csv
52601239.csv
556342.csv
48777788.csv
45365187.csv
8423286.csv
4324315.csv
68407464.csv
8278967.csv
402061.csv
31843000.csv
22492854.csv
12604499.csv
46313541.csv
45482553.csv
1174350.csv
3622246.csv
53869851.csv
2425874.csv
15876499.csv
338818.csv
2187788.csv
37862340.csv
43282.csv
43858028.csv
6803715.csv
42190461.csv
6754268.csv
2129507.csv
15263646.csv
13770757.csv
2637210.csv
10700938.csv
14449877.csv
22492840.csv
6241486.csv
509652.csv
26368.csv
2192391.csv
35397160.csv
65013168.csv
32713030.csv
7021949.csv
19160022.csv
21786305.csv
39657272.csv
15277155.csv
20112364.csv
35732059.csv
4013515.csv
1860229.csv
24439193.csv
23987535.csv
42475183.csv
38512332.csv
14432590.csv
1434568.csv
309384.csv
29255211.csv
40482414.csv
4303033.csv
11387962.csv
45506860.csv
1994357.csv
3263012.csv
42545732.csv
23936103.csv


...

Blank type: ****
Expecting value: line 1 column 1 (char 0)
43765315.csv
39517511.csv
14789914.csv
30773524.csv
45260466.csv
604757.csv
2192708.csv
12036216.csv
155964.csv
15532552.csv
4527747.csv
38625801.csv
33799446.csv
1918092.csv
51386083.csv
14771922.csv
1201311.csv
2674029.csv
22990712.csv
27705096.csv
4406945.csv
51390187.csv
10446419.csv
2104959.csv
50867792.csv
5116322.csv
34447325.csv
49252932.csv
38978851.csv
28140672.csv
1329489.csv
63007187.csv
14609655.csv
325591.csv
21326050.csv
57021678.csv
49434620.csv
2979882.csv
44498250.csv
40244497.csv
2170014.csv
5092744.csv
2290158.csv
16941465.csv
427863.csv
23249588.csv
168429.csv
41228670.csv
1944935.csv
59844454.csv
11004938.csv
64830694.csv
417778.csv
19519416.csv
7956701.csv
5421779.csv
11886823.csv
22953995.csv
1050858.csv
66847503.csv
7166916.csv
33711168.csv
4702557.csv
66278653.csv
317537.csv
31272.csv
1484140.csv
4540910.csv
4399580.csv
26885830.csv
37034336.csv
7608516.csv
13825715.csv
6224122.csv
235972.csv
5117766.c

/tmp/ipykernel_1879/3683733935.py:21: FutureWarning: The default value of regex will change from True to False in a future version.
  df_master['id'] = df_master['id'].str.replace('.csv','')


2542480.csv
40263821.csv
35247492.csv
21591304.csv
3900107.csv
15401991.csv
8423025.csv
7235299.csv
7275865.csv
1238433.csv
38307738.csv
4451000.csv
55659530.csv
53709923.csv
7178447.csv
5014482.csv
32979321.csv
1976818.csv
52120233.csv
26208295.csv
12095422.csv
13838195.csv
65283748.csv
1882755.csv
151451.csv
18199531.csv
47522465.csv
54994514.csv
10774598.csv
11705048.csv
7341524.csv
2595496.csv
19800337.csv
25340971.csv
36005413.csv
712041.csv
3954218.csv
43292378.csv
410188.csv
8524776.csv
12800045.csv
6859690.csv
64916900.csv
20167564.csv
57019278.csv
16977048.csv
22677427.csv
18880178.csv
11868865.csv
2461503.csv
34041428.csv
16834617.csv
4726218.csv
49267802.csv
19992127.csv
47473158.csv
47893431.csv
1320240.csv
19994542.csv
63247490.csv
16458436.csv
17661132.csv
46584821.csv
56848560.csv
24008332.csv
29187831.csv
164398.csv
3440828.csv
1651128.csv
29975573.csv
21625143.csv
10554828.csv
2018545.csv
2390979.csv
59385551.csv
2634592.csv
15879535.csv
4359149.csv
68269379.csv
151029

/tmp/ipykernel_1879/3683733935.py:21: FutureWarning: The default value of regex will change from True to False in a future version.
  df_master['id'] = df_master['id'].str.replace('.csv','')


Blank type: ****
Expecting value: line 1 column 1 (char 0)
206214.csv
1503167.csv
353229.csv
22617476.csv
472916.csv
217319.csv
50202690.csv
24759483.csv
40340961.csv
2306072.csv
1479807.csv
292461.csv
876135.csv
17495976.csv
16819020.csv
11610492.csv
42249696.csv
22673682.csv
3173768.csv
7951654.csv
23782546.csv
41777975.csv
21965195.csv
22785100.csv
534957.csv
42561433.csv
11117818.csv
66558684.csv
10100095.csv
49891985.csv
46244889.csv
511533.csv
5095163.csv
16650005.csv
1960713.csv
23692033.csv
45583405.csv
40159536.csv
517142.csv
52378745.csv
2483551.csv


In [ ]:
df_empty_type.info()
df_empty_type.head()
df_empty_people = df_empty_type[df_empty_type['types'].isin(entityPlaceTypesExt)]
df_empty_people.info()
df_empty_people.head(28)
# df_empty_type.head(28)

In [5]:
# REVIEW  cache files headers run just once, if needed
# FORMATING SOME FILES WITH ERRORS
files_list = [f for f in os.listdir('cacheSpotlightResponse') if not f.startswith('.')]
# parse to dataframe
df_files = pd.DataFrame(files_list, columns=['file_name'])
# df_files = df_files.query("file_name=='3075.csv'")
df_files.to_csv('temp_cacheFile.csv',index=False)
print(len(df_files))

for chunk in pd.read_csv('temp_cacheFile.csv', chunksize=30):
    df_files = pd.DataFrame()
    df_files['file_name'] = chunk['file_name']
    
    # iterate over files
    for file_name_item in df_files.itertuples():
        print(file_name_item.file_name)
        
        try:
            entitiesByBiography_df = pd.read_csv('cacheSpotlightResponse/'+file_name_item.file_name)

            headers = list(entitiesByBiography_df)
            change = False
            # Cleaning headers:
            # Case 1: headers still have character '@'
            if '@URI' in headers:
                change = True
                print("found 1")

            if '@surfaceForm' in headers:
                change = True
                print("found 2")
            # Case 2: still using surfaceForm instead of entity
            if 'surfaceForm' in headers:
                change = True
                print("found 3")
            # Case 3: no headers

            headers = [col.replace('@','') for col in headers]
            headers = [col.replace('surfaceForm','entity') for col in headers]

            if change:
                print(headers)
                entitiesByBiography_df.columns.values[0:11] = headers
                entitiesByBiography_df.to_csv('cacheSpotlightResponse/'+file_name_item.file_name,index=False)
        except Exception as e: 
            print("ERROR reading file: " + file_name_item.file_name)
            print(e)
            # # open CSV file and assign header
            # with open('cacheSpotlightResponse/'+file_name_item.file_name, 'wb') as file:
            #     dw = csv.DictWriter(file, delimiter=',',fieldnames=cols)
            #     dw.writeheader()

80242
1000228.csv
100273.csv
100487.csv
10085.csv
1009725.csv
ERROR reading file: 1009725.csv
Error tokenizing data. C error: Expected 7 fields in line 171, saw 11

1010510.csv
ERROR reading file: 1010510.csv
Error tokenizing data. C error: Expected 7 fields in line 578, saw 11

1010943.csv
ERROR reading file: 1010943.csv
Error tokenizing data. C error: Expected 7 fields in line 29, saw 11

10120.csv
1013900.csv
ERROR reading file: 1013900.csv
Error tokenizing data. C error: Expected 7 fields in line 490, saw 11

1022191.csv
ERROR reading file: 1022191.csv
Error tokenizing data. C error: Expected 7 fields in line 151, saw 11

1023303.csv
ERROR reading file: 1023303.csv
Error tokenizing data. C error: Expected 7 fields in line 80, saw 11

1024347.csv
ERROR reading file: 1024347.csv
Error tokenizing data. C error: Expected 7 fields in line 99, saw 11

1028178.csv
ERROR reading file: 1028178.csv
Error tokenizing data. C error: Expected 7 fields in line 1145, saw 11

103549.csv
ERROR readi

In [15]:
# 1. Extract People and Places entities
entitiyTypes = ['DBpedia:Person','DBpedia:MusicalArtist','DBpedia:Place','DBpedia:SocietalEvent']
# http://dbpedia.org/ontology/Person
# df_entityType = pd.DataFrame({'id':[1,2],'types':['http://dbpedia.org/ontology/Person',
#                                                       'http://dbpedia.org/ontology/MusicalArtist']})
# read master with index of entities
df_master_cache = pd.read_csv('cacheSpotlightResponse/emptyTypes_master.csv')

# use chunk to load a small number of files in memory
for chunk in pd.read_csv('totalBiographiesEntities.csv', chunksize=30):
    df_files = pd.DataFrame()
    df_files['file_name'] = chunk['file_name']
    
    # iterate over files
    for file_name_item in df_files.itertuples():
        try:
            # start = time.time()
            print(file_name_item.file_name)
            # check if the file exists, e.g., 10085.csv
            # file should exist, it was create during queryDBpedia process
            file_exists = os.path.isfile('cacheSpotlightResponse/'+file_name_item.file_name)

            if file_exists:
                # read the cached results from the query
                entitiesByBiography_df = pd.read_csv('cacheSpotlightResponse/'+file_name_item.file_name)
                # surfaceForm: entity as found in the text
                # entitiesByBiography_df.rename(columns = {'surfaceForm':'entity'},inplace = True)
                # column types contain DBpedia classes, e.g., dbo:Person, Schema:Place, etc.
                # select all rows where types <> NA, not empty = ne
                df_ne = entitiesByBiography_df.loc[~entitiesByBiography_df['types'].isna()] 

                df_result = pd.DataFrame()
                if len(df_ne) >0:
                    for entity in entitiyTypes:
                        df_temp = df_ne[df_ne['types'].str.contains(entity)].copy()

                        if not df_temp.empty:
                            if entity == 'DBpedia:Person' or entity == 'DBpedia:MusicalArtist':
                                df_temp['entType'] = 'person'
                            elif entity == 'DBpedia:Place':
                                df_temp['entType'] = 'place'
                            elif entity == 'DBpedia:SocietalEvent' or entity == 'DBpedia:Event':
                                df_temp['entType'] = 'event'

                            df_result = df_result.append(df_temp)

                # now filter all the entities that are empty, this df contains all the info of the matching
                # e.g., surfaceForm, similarityScore, etc
                df_e = entitiesByBiography_df.loc[entitiesByBiography_df['types'].isna()] 
                
                # only if the dataframe has some rows with empty value in column "entity"
                if not df_e.empty:
                    # list of entities and the name of the files that store the results from the query
                    # when merging, only the entities that are stored in cache will have a file name value in column "file_name"
                    df_merge = df_master_cache.merge(df_e, on=['URI','entity'],how='right')
                    # df_merge.to_csv('extractedEntitiesPersonPlaceOnly/df_merge.csv',index=False)

                    # for all documents not in cache, meaning that value in column "file_name" is empty
                    df_cache = df_merge.loc[df_merge['file_name'].isna()]
                    print(len(df_cache))
                    if len(df_cache)>0:
                        # iterate over the df with all the entities that not in cache
                        for item_empty in df_cache.itertuples():
                            # query DBpedia, store results in local folder and update the master file
                            df_temp = queryEntityLeft(item_empty.URI,item_empty)
                        
                        # read the master cache again with the latest updates after checking for URIs in previous line
                        df_master_cache = pd.read_csv('cacheSpotlightResponse/emptyTypes_master.csv')
                        # merge again all the entities that had column "types" value as empty to find the name of the file
                        # where the response from DBpedia is stored
                        df_merge = df_master_cache.merge(df_e, on=['URI','entity'],how='right')
                    # df_merge.to_csv('extractedEntitiesPersonPlaceOnly/df_merge.csv',index=False)

                    # df_cache will contain all the entities and the file_name value from where to read its URIs
                    # filter all the rows that have column "file_name" as having a value
                    df_cache = df_merge.loc[~df_merge['file_name'].isna()] 
                    for item_queriedType in df_cache.itertuples():
                        # print(item_queriedType.entity)
                        appendRow = False
                        typeEntity = ''

                        # read cache file
                        file_exists = os.path.isfile('cacheSpotlightResponse/'+item_queriedType.file_name)
                        if file_exists:
                            df_empty_type = pd.read_csv('cacheSpotlightResponse/'+item_queriedType.file_name)
                            df_empty_type.drop_duplicates(subset=['entity','URI','types'],keep='first')
                            
                            # once the file is located, filter the rows that match entity and URI
                            try:
                                df_empty_type = df_empty_type.query("""entity == "{}" and URI=="{}" """.format(item_queriedType.entity,item_queriedType.URI))
                            except SyntaxError as ex:
                                # df_empty_type = df_empty_type.query("""entity == '{}' and URI=='{}'""".format(item_queriedType.entity,item_queriedType.URI))
                                df_empty_type = df_empty_type[df_empty_type['URI'].str.contains(item_queriedType.URI)]
                            except TokenError as te:
                                df_empty_type = df_empty_type.query("entity == '{}' and URI=='{}'".format(item_queriedType.entity,item_queriedType.URI))

                            # Classify entities according to type
                            # First, identify entity types Place, filter all rows with column "types" value equal to Place URI
                            # print(df_temp.URI)
                            # print(df_temp.types)
                            
                            df_temp = df_empty_type[df_empty_type['types'].str.contains('http://dbpedia.org/ontology/Place')].copy()
                            if not df_temp.empty:
                                types = df_temp['types'].loc[df_temp.index[0]]
                                entType = 'place'
                                appendRow = True
                            # Second, identify entity types Person, filter all rows with column "types" value equal to Person URI
                            df_temp = df_empty_type.query('types == "http://dbpedia.org/ontology/Person" or types=="http://dbpedia.org/ontology/MusicalArtist"')
                            df_temp = df_temp.sort_values(by='types', ascending=False)
                            
                            if not df_temp.empty:
                                # Before adding this entity to the list, review that it is a person and it is not a PersonFunction or MusicalTerminology
                                # call function, search all the values in DBpedia, returs True if the URI is of a person, False otherwise
                                response = checkPersonEntity(item_queriedType.URI)
                                if not response:
                                    appendRow = False
                                    continue
                                types = df_temp['types'].loc[df_temp.index[0]]
                                entType = 'person'
                                appendRow = True
                            # else:
                            #     print(item_queriedType.URI)
                            #     print("Empty: " + item_queriedType.file_name)

                            if appendRow:
                                df_new_row = pd.DataFrame([[item_queriedType.URI, item_queriedType.support, types, item_queriedType.entity, item_queriedType.offset, 
                                                            item_queriedType.similarityScore,item_queriedType.percentageOfSecondRank, item_queriedType.sentence, 
                                                            item_queriedType.sentenceIndex, item_queriedType.paragraphIndex, 
                                                           item_queriedType.section]],columns=list(entitiesByBiography_df))
                                df_new_row['entType'] = entType
                                df_result = df_result.append(df_new_row,ignore_index=True)
                        else:
                            print(df_merge.file_name)
                            print(df_merge.URI)
                            break

                    # print('No empty types')
            # IF file does not exists, then the 
            else:
                print("[INFO] - File does not exist, run 020_queryDbpedia book. File: " + file_name_item.file_name)

            # df_result['wikiPageID'] = file_name_item.file_name.replace('.csv','')
            df_result.drop(['sentence','wikiPageID','section'], axis=1,inplace=True)
            df_result.drop_duplicates(subset=['paragraphIndex','sentenceIndex','offset','entity','URI'],keep='first',inplace=True)
            
            df_result.to_csv('extractedEntitiesPersonPlaceOnly/'+file_name_item.file_name,index=False)
            # end = time.time()
            # print("The time of execution of above program is :", end-start)
        except (pd.errors.ParserError, KeyError) as pe:
            print('Error: ' + file_name_item.file_name)
            if hasattr(pe, 'message'):
                print(pe.message)
            else:
                print(pe)
            df_error = pd.DataFrame([[file_name_item.file_name]],columns=['file_name'])
            df_error.to_csv('temp_error.csv',mode='a',index=False,header=False)
    time.sleep(120)

1009725.csv
Error: 1009725.csv
Error tokenizing data. C error: Expected 7 fields in line 171, saw 11

1010510.csv
Error: 1010510.csv
Error tokenizing data. C error: Expected 7 fields in line 578, saw 11

1010943.csv
Error: 1010943.csv
Error tokenizing data. C error: Expected 7 fields in line 29, saw 11

1013900.csv
Error: 1013900.csv
Error tokenizing data. C error: Expected 7 fields in line 490, saw 11

1022191.csv
Error: 1022191.csv
Error tokenizing data. C error: Expected 7 fields in line 151, saw 11

1023303.csv
Error: 1023303.csv
Error tokenizing data. C error: Expected 7 fields in line 80, saw 11

1024347.csv
Error: 1024347.csv
Error tokenizing data. C error: Expected 7 fields in line 99, saw 11

1028178.csv
Error: 1028178.csv
Error tokenizing data. C error: Expected 7 fields in line 1145, saw 11

103549.csv
Error: 103549.csv
Error tokenizing data. C error: Expected 7 fields in line 336, saw 11

103566.csv
Error: 103566.csv
Error tokenizing data. C error: Expected 7 fields in line

/tmp/ipykernel_296/1429907359.py:21: FutureWarning: The default value of regex will change from True to False in a future version.
  df_master['id'] = df_master['id'].str.replace('.csv','')


3136059.csv
0
313835.csv
2
3141790.csv
1
3143897.csv
2
314487.csv
10
3152159.csv
0
3155747.csv
0
316711.csv
0
3180265.csv
0
3180885.csv
0
3190132.csv
1
319074.csv
11
3197212.csv
0
3204508.csv
0
320685.csv
4
3212385.csv
1
3217228.csv
0
322567.csv
0
3225681.csv
0
3227606.csv
1
3228328.csv
0
323499.csv
3
3236079.csv
0
3236194.csv
2
325790.csv
0
3263983.csv
0
3266294.csv
0
3273.csv
[INFO] - File does not exist, run 020_queryDbpedia book. File: 3273.csv
3280230.csv
0
328330.csv
1
328940.csv
0
3291750.csv
0
329722.csv
0
330103.csv
0
3301109.csv
1
3302723.csv
11
331463.csv
22
3337272.csv
2
3344033.csv
4
3347663.csv
3
3358257.csv
2
3365880.csv
3
3372135.csv
0
3373682.csv
0
3388232.csv
3
3391490.csv
1
3394339.csv
2
3396268.csv
0
340945.csv
0
3411455.csv
0
3418089.csv
0
341837.csv
2
3419345.csv
0
343176.csv
0
3445142.csv
0
344562.csv
0
3450382.csv
0
3450589.csv
1
345114.csv
6
3452637.csv
1
3460078.csv
1
346303.csv
0
3463240.csv
4
347201.csv
0
3478434.csv
0
348348.csv
0
3488554.csv
2
3496560.csv


/tmp/ipykernel_296/3551301871.py:93: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  df_empty_type = df_empty_type[df_empty_type['URI'].str.contains(item_queriedType.URI)]


67892.csv
3
679806.csv
0
68308.csv
0
690900.csv
0
695088.csv
0
695584.csv
1
696261.csv
0
696825.csv
0
699151.csv
0
70020.csv
1
701860.csv
0
702942.csv
0
70315.csv
2
703488.csv
0
706650.csv
0
707930.csv
0
708521.csv
1
709620.csv
0
70986.csv
3
711489.csv
1
712322.csv
1
715167.csv
0
716617.csv
0
719979.csv
0
720273.csv
0
720783.csv
7
722501.csv
4
726151.csv
2
728429.csv
6
734633.csv
0
735176.csv
0
736528.csv
6
739770.csv
0
741791.csv
0
742176.csv
0
746774.csv
1
747500.csv
2
75200.csv
2
752694.csv
1
754282.csv
1
756836.csv
0
757387.csv
0
7585.csv
3
759988.csv
1
760333.csv
0
761492.csv
0
762314.csv
1
76252.csv
5
763791.csv
1
766300.csv
1
7672.csv
2
767288.csv
1
770812.csv
0
772715.csv
0
777352.csv
0
777623.csv
0
78231.csv
2
782651.csv
0
787927.csv
0
787997.csv
0
788959.csv
0
790333.csv
0
793333.csv
0
795427.csv
1
797572.csv
1
806373.csv
0
80696.csv
6
8095.csv
4
817466.csv
0
820730.csv
1
82413.csv
3
825643.csv
3
827409.csv
0
827624.csv
0
8300.csv
0
833109.csv
7
833172.csv
5
8351.csv
4
837456

KeyboardInterrupt: 

In [4]:
# Reprocess data to delete information not needed.
# use chunk to load a small number of files in memory
# list_wikiIdSample
# totalBiographiesBenchmark
for chunk in pd.read_csv('list_wikiIdSample.csv', chunksize=30):
# for chunk in pd.read_csv('totalBiographiesBenchmark.csv', chunksize=30):
    df_files = pd.DataFrame()
    df_files['file_name'] = chunk['file_name']
    
    # iterate over files
    for file_name_item in df_files.itertuples():
        try:
            # start = time.time()
            print(file_name_item.file_name)
            # check if the file exists, e.g., 10085.csv
            # file should exist, it was create during queryDBpedia process
            file_exists = os.path.isfile('extractedEntitiesPersonPlaceOnly/'+file_name_item.file_name)

            if file_exists:
                df_pp = pd.read_csv('extractedEntitiesPersonPlaceOnly/'+file_name_item.file_name)
                df_pp.drop(['sentence','wikiPageID','section'], axis=1,inplace=True)
                df_pp.drop_duplicates(subset=['paragraphIndex','sentenceIndex','offset','entity','URI'],keep='first',inplace=True)
                df_pp.to_csv('extractedEntitiesPersonPlaceOnly/'+file_name_item.file_name,index=False)
            else:
                print("[INFO] - File does not exist, run 020_queryDbpedia book. File: " + file_name_item.file_name)
        except (pd.errors.ParserError, KeyError) as pe:
            print('Error: ' + file_name_item.file_name)
            if hasattr(pe, 'message'):
                print(pe.message)
            else:
                print(pe)

608845.csv
1048151.csv
2232977.csv
1913885.csv
1790990.csv
409969.csv
1709886.csv
1551347.csv
50782750.csv
181946.csv
579599.csv
50963136.csv
226142.csv
1422240.csv
1174545.csv
70020.csv
752694.csv
312781.csv
2898019.csv
2253021.csv
3450382.csv
2553865.csv
3263983.csv
2320846.csv
144624.csv
827409.csv
50902387.csv
671637.csv
562392.csv
1396921.csv
2815597.csv
858538.csv
2269540.csv
1205991.csv
701860.csv
2334176.csv
252147.csv
652114.csv
320685.csv
8716.csv
3606266.csv
576282.csv
1232492.csv
113049.csv
43165.csv
3770842.csv
221191.csv
1491559.csv
1790137.csv
529161.csv
167975.csv
3126224.csv
78231.csv
1022191.csv
154038.csv
223497.csv
356414.csv
2625004.csv
1359335.csv
3081864.csv
2450365.csv
63747.csv
739770.csv
1566844.csv
50230.csv
439467.csv
181985.csv
2296293.csv
2520364.csv
2682932.csv
3302723.csv
51560453.csv
891378.csv
3236079.csv
1914666.csv
1593504.csv
152097.csv
720273.csv
419012.csv
51012206.csv
1234422.csv
994118.csv
1181499.csv
341837.csv
788959.csv
1394504.csv
3818460.cs